In [2]:
!pip3 install pyqt6

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 83.1 MB/s eta 0:00:006 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 46.3 MB/s eta 0:00:000m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.9/282.9 kB 32.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import sys
import xml.etree.ElementTree as ET
from PyQt5.QtWidgets import (QApplication, QMainWindow, QFileDialog, 
                             QVBoxLayout, QWidget, QPushButton, QTextEdit, 
                             QTreeWidget, QTreeWidgetItem, QLabel, QHBoxLayout)
from pybullet_utils import bullet_client as bc
import pybullet as p
import pybullet_data
from PyQt6.QtOpenGLWidgets import QOpenGLWidget
import OpenGL.GL as gl

class PyBulletViewer(QOpenGLWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.physics_client = None
        self.urdf_file = None

    def initializeGL(self):
        self.physics_client = bc.BulletClient(connection_mode=p.DIRECT)
        self.physics_client.setAdditionalSearchPath(pybullet_data.getDataPath())
        gl.glEnable(gl.GL_DEPTH_TEST)

    def load_urdf(self, urdf_file):
        if self.urdf_file:
            self.physics_client.resetSimulation()
        self.urdf_file = urdf_file
        self.physics_client.loadURDF(self.urdf_file)
        self.update()

    def paintGL(self):
        self.physics_client.stepSimulation()
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)


def parse_urdf(file_path):
    tree = ET.parse(file_path)
    return tree

class URDFEditor(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("URDF Editor")
        self.setGeometry(100, 100, 1200, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        layout = QHBoxLayout()
        self.central_widget.setLayout(layout)

        left_layout = QVBoxLayout()
        layout.addLayout(left_layout)
        
        self.load_button = QPushButton("Load URDF")
        self.load_button.clicked.connect(self.load_urdf)
        left_layout.addWidget(self.load_button)

        self.tree_widget = QTreeWidget()
        self.tree_widget.setHeaderLabels(["URDF Structure"])
        left_layout.addWidget(self.tree_widget)

        self.editor = QTextEdit()
        left_layout.addWidget(self.editor)
        
        self.save_button = QPushButton("Save URDF")
        self.save_button.clicked.connect(self.save_urdf)
        left_layout.addWidget(self.save_button)
        
        self.visualize_button = QPushButton("Visualize in PyBullet")
        self.visualize_button.clicked.connect(self.visualize_urdf)
        left_layout.addWidget(self.visualize_button)
        
        self.viewer = PyBulletViewer()
        layout.addWidget(self.viewer, 1)

        self.urdf_tree = None
        self.urdf_file = None

    def load_urdf(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open URDF File", "", "URDF Files (*.urdf)")
        if file_path:
            self.urdf_file = file_path
            self.urdf_tree = parse_urdf(file_path)
            self.populate_tree()
            self.editor.setPlainText(ET.tostring(self.urdf_tree.getroot(), encoding='utf8').decode('utf8'))
            self.viewer.load_urdf(file_path)
    
    def populate_tree(self):
        self.tree_widget.clear()
        root = self.urdf_tree.getroot()
        root_item = QTreeWidgetItem([root.tag])
        self.tree_widget.addTopLevelItem(root_item)
        self.add_children(root, root_item)

    def add_children(self, parent_xml, parent_item):
        for child in parent_xml:
            child_item = QTreeWidgetItem([child.tag])
            parent_item.addChild(child_item)
            self.add_children(child, child_item)

    def save_urdf(self):
        if self.urdf_tree:
            file_path, _ = QFileDialog.getSaveFileName(self, "Save URDF File", "", "URDF Files (*.urdf)")
            if file_path:
                with open(file_path, "w") as file:
                    file.write(self.editor.toPlainText())

    def visualize_urdf(self):
        if self.urdf_file:
            self.viewer.load_urdf(self.urdf_file)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    editor = URDFEditor()
    editor.show()
    sys.exit(app.exec())


pybullet build time: Nov 28 2023 23:51:11
